# Attribution

This notebook is a modified version of one shared with us by Martin Jones of the Francis Crick Institute. Thank you, Martin!

In [ ]:
# if needed, pip install the panoptes-client
#import sys
#!{sys.executable} -m pip install --user panoptes-client -U 

## Set up Zooniverse project information

Set variables that define the Zooniverse project and user and image directory containing jpegs to upload.

In [15]:
# unique zooniverse project id
PROJECT_ID = "h-spiers/etch-a-cell-mito-mega-mix"

# whether uploaded images are flipbooks of a few jpegs or a single jpeg
IS_FLIPBOOK = True

# only applies to flipbooks: number of
# images above and below the central slice
# of the flipbook
# e.g. SPAN = 2 is the central slice plus 2 from above and 2 from below (a total of 5 images)
# CAUTION: CHANGING THIS NUMBER MAY BREAK OTHER SCRIPTS!
SPAN = 2

# zooniverse username
USERNAME = "conradry"

# name of the subject set to be created
SUBJECT_SET_NAME = "Batch5a"

# whether this is a test if True, then
# subjects won't be uploaded to zooniverse
TESTING = False

# local directory containing jpeg images for upload
wdir = "../batch5/extra_images/"

## Upload images

Login to Zooniverse, create the new subject set, add images/flipbooks and upload!

In [9]:
from panoptes_client import Project, Panoptes, Subject, SubjectSet
import glob
import os
import sys
import getpass
import re
from tqdm import tqdm

In [10]:
# This function builds the subject from the chosen set of images and attaches metadata
def build_subject_flipbook(project, file_list, start_index, end_index):
    subject = Subject() # Inititialise a subject
    subject.links.project = project # ...attach it to a project
    subject.metadata['Subject ID'] = start_index + SPAN # Add the names of the images
    
    # For loop to attach the images to the subject one-by-one
    for i, idx in enumerate(range(start_index, end_index)):
        fname = str(file_list[idx])
        subject.add_location(fname)
        subject.metadata['Image %d' % i] = os.path.basename(fname)
        
    subject.metadata['default_frame'] = SPAN + 1  # We want people to annotate the middle image
    
    # Metadata from here should be changed according to the data
    # any keys are allowed for the metadata
    subject.metadata['Microscope'] = 'VolumeEM'
    subject.metadata['Raw XY resolution (nm)'] = 1
    subject.metadata['Raw Z resolution (nm)'] = 1
    subject.metadata['Scaling factor'] = 1
    subject.metadata['jpeg quality (%)'] = 100
    subject.metadata['Attribution'] = 'CEM1.5M'
    subject.metadata['Description'] = 'CEM1.5M_Batch_Mitos'
    print("Starting to save")
    print(subject)
    subject.save()
    print("Subject saved")

    return subject

def build_subject_image(project, file_list, index):
    subject = Subject() # Inititialise a subject
    subject.links.project = project # ...attach it to a project
    subject.metadata['Subject ID'] = index
    
    # For loop to attach the images to the subject one-by-one
    fname = str(file_list[index])
    subject.add_location(fname)
    subject.metadata['Image 0'] = os.path.basename(fname)
        
    #subject.metadata['default_frame'] = 0  # We want people to annotate the middle image
    
    # Metadata from here should be changed according to the data
    # any keys are allowed for the metadata
    subject.metadata['Microscope'] = 'TEM'
    subject.metadata['Raw XY resolution (nm)'] = 1
    subject.metadata['Raw Z resolution (nm)'] = 1
    subject.metadata['Scaling factor'] = 1
    subject.metadata['jpeg quality (%)'] = 100
    subject.metadata['Attribution'] = 'CEM1.5M'
    subject.metadata['Description'] = 'CEM1.5M_Batch_Mitos'
    print("Starting to save")
    print(subject)
    subject.save()
    print("Subject saved")

    return subject

This function connects to the Zooniverse, make sure the Project ID and username were set correctly earlier.

In [11]:
def connect_to_zooniverse(project_id=PROJECT_ID, user_name=USERNAME):
    try:
        password = getpass.getpass(prompt='Password: ', stream=None)
        Panoptes.connect(username=user_name, password=password)
        print("Connected to Zooniverse")
    except Exception as e:
        print("Couldn't connect to Zooniverse")
        print("Exception {}".format(e))
        sys.exit(1)
        
    print(f"Connecting to {project_id}...")
    project = Project.find(slug=project_id)
    print("...connected!")
    return project

In [12]:
# Helper function to initialise a "subject set" and attach it to a project
def initialise_subject_set(project, subject_name):
    subject_set = SubjectSet()
    subject_set.links.project = project
    subject_set.display_name = subject_name
    subject_set.save()
    return subject_set

# Function to build a subject set from a fixed range of images
def build_subject_set_flipbook(project, file_list, file_idx_start, file_idx_stop):
    print(f"project {project}\n", 
          f"file_idx_start {file_idx_start}\n", 
          f"file_idx_stop {file_idx_stop}\n"
         )
    
    print(f"Building subject set from files {file_idx_start}-{file_idx_stop}")
    subjects = []
    
    subject = build_subject_flipbook(project, file_list, file_idx_start, file_idx_stop)
    subjects.append(subject)
    
    return subjects

def build_subject_set_image(project, file_list, file_idx):
    print(f"project {project}\n", 
          f"file_idx {file_idx}\n"
         )
    
    print(f"Building subject set from file {file_idx}")
    subjects = []

    subject = build_subject_image(project, file_list, file_idx)
    subjects.append(subject)
    
    return subjects

Connect to the Zooniverse using your credentials

In [6]:
project = connect_to_zooniverse(project_id=PROJECT_ID)

Password: ········
Connected to Zooniverse
Connecting to h-spiers/etch-a-cell-mito-mega-mix...
...connected!


In [16]:
# N.B. this will work for directories populated with jpg images
# by either the prep_stacks.py or prep_images.py script

file_list = sorted(glob.glob(os.path.join(wdir, '*.jpg')))
n_files = len(file_list)

file_basenames = [os.path.basename(file) for file in file_list]
if IS_FLIPBOOK:
    file_basename_prefixes = [file.split('_zloc')[0] for file in file_basenames]
    indices = list(range(0, len(file_list) + 1, 5))
    print('num flipbooks:', n_files / 5)
else:
    file_basename_prefixes = [file.split('.jpg')[0] for file in file_basenames]
    indices = list(range(0, len(file_list)))
    print('num images:', n_files)

num flipbooks: 15.0


**Note: this can be a very slow process to upload!**

In [ ]:
# create subject set of name defined above
print(f"Creating subject set name {SUBJECT_SET_NAME}\n")
subject_set = initialise_subject_set(project, SUBJECT_SET_NAME)

In [20]:
# add images/flipbooks as subjects to the subject set
subjects = []
if IS_FLIPBOOK:
    for counter, (start_index, end_index) in enumerate(zip(indices[:-1], indices[1:])):
        print(f"\n*******\nStep {counter}")
        subjects.append(build_subject_set_flipbook(project, file_list, start_index, end_index))
else:
    for counter, index in enumerate(indices):
        print(f"\n*******\nStep {counter}")
        subjects.append(build_subject_set_image(project, file_list, index))


*******
Step 0
project <Project 15805>
 file_idx_start 0
 file_idx_stop 5

Building subject set from files 0-5
Starting to save
<Subject None>
Subject saved

*******
Step 1
project <Project 15805>
 file_idx_start 5
 file_idx_stop 10

Building subject set from files 5-10
Starting to save
<Subject None>
Subject saved

*******
Step 2
project <Project 15805>
 file_idx_start 10
 file_idx_stop 15

Building subject set from files 10-15
Starting to save
<Subject None>
Subject saved

*******
Step 3
project <Project 15805>
 file_idx_start 15
 file_idx_stop 20

Building subject set from files 15-20
Starting to save
<Subject None>
Subject saved

*******
Step 4
project <Project 15805>
 file_idx_start 20
 file_idx_stop 25

Building subject set from files 20-25
Starting to save
<Subject None>
Subject saved

*******
Step 5
project <Project 15805>
 file_idx_start 25
 file_idx_stop 30

Building subject set from files 25-30
Starting to save
<Subject None>
Subject saved

*******
Step 6
project <Project 1

In [21]:
# upload the subject sets to the Zooniverse
if not TESTING:
    for subj in tqdm(subjects):
        subject_set.add(subj)

100%|██████████| 15/15 [00:02<00:00,  7.24it/s]


## Add subject IDs

Optionally add subject ids to each subjects metadata. This makes it possible for annotators to lookup and reference particular subjects for later review.

In [17]:
# load all subject sets in project and print their metadata
subject_sets = SubjectSet.where(scope='project', project_id=project.id)
ssets = list(subject_sets)

for sset in ssets:
    print(sset.raw)

{'display_name': 'Batch0', 'links': {'project': '15805', 'workflows': ['18276'], 'subjects': []}, 'metadata': {}, 'id': '95613', 'set_member_subjects_count': 41, 'created_at': '2021-06-04T12:52:47.940Z', 'updated_at': '2021-06-04T12:55:51.704Z', 'href': '/subject_sets/95613', 'completeness': {}}
{'display_name': 'Batch2a', 'links': {'project': '15805', 'workflows': ['19199'], 'subjects': []}, 'metadata': {}, 'id': '96898', 'set_member_subjects_count': 700, 'created_at': '2021-07-23T16:38:06.557Z', 'updated_at': '2021-07-23T16:45:37.546Z', 'href': '/subject_sets/96898', 'completeness': {}}
{'display_name': 'Batch2b', 'links': {'project': '15805', 'workflows': ['19378'], 'subjects': []}, 'metadata': {}, 'id': '96899', 'set_member_subjects_count': 665, 'created_at': '2021-07-23T16:52:43.070Z', 'updated_at': '2021-07-23T16:59:01.663Z', 'href': '/subject_sets/96899', 'completeness': {}}
{'display_name': 'Batch3a', 'links': {'project': '15805', 'workflows': ['19464'], 'subjects': []}, 'metad

In [18]:
subject_set = ssets[-2]

In [19]:
subject_set

<SubjectSet 99927>

In [ ]:
# get the subjects for one of the subject sets
# (index 0 for illustration only)
subjects = list(ssets[0].subjects)

In [ ]:
# add ID key to the subject metadata
#for subject in tqdm(subjects):
#    subject.metadata['ID'] = subject.raw['id']
#    subject.save()

## Drop subject

Delete subjects that were created accidentally.

In [ ]:
# load all subject sets in project and print their metadata
subject_sets = SubjectSet.where(scope='project', project_id=project.id)
ssets = list(subject_sets)

for sset in ssets:
    print(sset.raw)

In [ ]:
# get the subjects for one of the subject sets
# (index 0 for illustration only)
subjects = list(ssets[0].subjects)

In [ ]:
# delete the subjects!! use with care!!
#for subject in tqdm(subjects):
#    subject.delete()